In [52]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.discriminant_analysis import StandardScaler
from sklearn.linear_model import LogisticRegression

from sklearn.feature_selection import SelectKBest


In [53]:
df = pd.read_csv("../datasets/insurance.csv")
# missing values
print("--------------------")
print("Missing values:")
print(df.isnull().sum())
print("--------------------")

print(df.describe())


--------------------
Missing values:
age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64
--------------------
               age          bmi     children       charges
count  1338.000000  1338.000000  1338.000000   1338.000000
mean     39.207025    30.663397     1.094918  13270.422265
std      14.049960     6.098187     1.205493  12110.011237
min      18.000000    15.960000     0.000000   1121.873900
25%      27.000000    26.296250     0.000000   4740.287150
50%      39.000000    30.400000     1.000000   9382.033000
75%      51.000000    34.693750     2.000000  16639.912515
max      64.000000    53.130000     5.000000  63770.428010


In [54]:
#String to numerical values
df['sex'] = df['sex'].map({'male': 1, 'female': 0})
df['smoker'] = df['smoker'].map({'yes': 1, 'no': 0})
df['region'] = df['region'].map({'northeast': 1, 'southeast': 2, 'southwest': 3, 'northwest': 4})
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,0,27.900,0,1,3,16884.92400
1,18,1,33.770,1,0,2,1725.55230
2,28,1,33.000,3,0,2,4449.46200
3,33,1,22.705,0,0,4,21984.47061
4,32,1,28.880,0,0,4,3866.85520


In [55]:
# Q1 and Q2
Q1 = df['charges'].quantile(0.25)
Q3 = df['charges'].quantile(0.75)

#interquartile range
IQR = Q3 - Q1

# identify outliers
lower_bound = Q1 - 1.5*IQR
upper_bound = Q3 + 1.5*IQR
outliers = df[(df['charges'] < lower_bound) | (df['charges'] > upper_bound)]

df = df.drop(outliers.index)

In [56]:
MinMaxscaler=MinMaxScaler()
df.insert(0, 'id', range(len(df)))
id = df.pop('id')
df_scaled = pd.DataFrame(MinMaxscaler.fit_transform(df), columns=df.columns)
df = pd.concat([df_scaled, id.reset_index(drop=True)], axis=1)
last_col = df.pop(df.columns[-1])
df.insert(0, last_col.name, last_col)
df.head()

,id,age,sex,bmi,children,smoker,region,charges
0,0,0.021739,0.0,0.321227,0.0,1.0,0.666667,0.472641
1,1,0.000000,1.0,0.479150,0.2,0.0,0.333333,0.018101
2,2,0.217391,1.0,0.458434,0.6,0.0,0.333333,0.099775
3,3,0.326087,1.0,0.181464,0.0,0.0,1.000000,0.625547
4,4,0.304348,1.0,0.347592,0.0,0.0,1.000000,0.082306


### Regression Model

In [57]:
# test:30% train:70%
X = df.drop('charges', axis=1)
y = df['charges']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [ ]:
#develop model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
y_pred = lr_model.predict(X_test)

#evaluate
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("Root Mean Squared Error(RMSE):",rmse)
print("Mean Squared Error(MSE):", mse)
print("R-squared (coefficient of determination):",r2)

# plot graph
plt.scatter(y_test, y_pred)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k-')
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Predicted vs. Actual House Prices')
plt.show()

print("Accuracy of the model: {:.2f}%".format(lr_model.score(X_test, y_test)*100))

### SVM

In [61]:

from sklearn import svm

clf = svm.SVC(kernel='linear') # Linear kernel

clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

# import scikit-learn metrics modue for accuracy calculation
from sklearn import metrics
from sklearn.metrics import r2_score


# Model Accuracy : how often is the classifier correct?
print("Accuracy",metrics.accuracy_score(y_test,y_pred))

ValueError: Unknown label type: 'continuous'